In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from captum.attr import IntegratedGradients

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from transformers import BertTokenizer, BertForSequenceClassification, BertConfig

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from sklearn.model_selection import train_test_split
from spacy.lang.en import English
from splitbert.textsplit import text_segmentation
from splitbert.splitbert import SplitBertConcatEncoderModel
from splitbert.splitbert import conduct_input_ids_and_attention_masks
from splitbert.splitbert import make_masks
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import tqdm

/home/mykim/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
2023-07-18 11:23:29.476562: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


# Data Preparation

In [2]:
post_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_post.csv', encoding='UTF-8')
comment_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_comment.csv', encoding='UTF-8')
reply_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/avg_satisfaction_raw_0-999.csv', encoding='ISO-8859-1')

modes = [['seg', 'seg', 'snt']]

nlp = English()
nlp.add_pipe("sentencizer")

# satisfaction score (y)
satisfactions_float = list(reply_df['satisfy_composite'])
satisfactions = []

for s in satisfactions_float:
    if s < 3.5:
        satisfactions.append(0)
    elif s < 5:
        satisfactions.append(1)
    else:
        satisfactions.append(2)

reply_contents = list(reply_df['replyContent'])
post_contents = list(post_df['content'])
comment_bodies = list(comment_df['content'])


def get_sequences(contents, mode):
    sequences = []

    if mode == 'all':
        for content in contents:
            sequences.append([content])
    elif mode == 'seg':
        for content in contents:
            sentences = list(map(lambda x: str(x), list(nlp(content).sents)))
            sequences.append(text_segmentation(sentences))
    else:  # sentences
        for content in contents:
            sequences.append(list(map(lambda x: str(x), list(nlp(content).sents))))

    return sequences


for mode in modes:
    print(mode)
    post_sequences = get_sequences(post_contents, mode[0])
    comment_sequences = get_sequences(comment_bodies, mode[1])
    reply_sequences = get_sequences(reply_contents, mode[2])

    data = []
    max_post, max_comment, max_reply = 0, 0, 0
    i = 0
    for post, comment, reply, satisfaction, satisfaction_float in zip(post_sequences, comment_sequences,
                                                                          reply_sequences, satisfactions,
                                                                          satisfactions_float):
        if len(post) > max_post:
            max_post = len(post)
        if len(comment) > max_comment:
            max_comment = len(comment)
        if len(reply) > max_reply:
            max_reply = len(reply)

        data.append([i, post, comment, reply, satisfaction, satisfaction_float])
        i += 1

    print(max_post, max_comment, max_reply)
    max_count = max(max_post, max_comment, max_reply)
    print(max_count)

    columns = ['index', 'post_contents', 'comment_contents', 'reply_contents', 'label', 'score']
    df = pd.DataFrame(data, columns=columns)

    # data split (train & test sets)
    idx_train, idx_remain = train_test_split(df.index.values, test_size=0.20, random_state=42)
    idx_val, idx_test = train_test_split(idx_remain, test_size=0.50, random_state=42)

    train_df = df.iloc[idx_train]
    val_df = df.iloc[idx_val]
    test_df = df.iloc[idx_test]

    count_min_label = min(train_df['label'].value_counts())

    labels = [0, 1, 2]

    train_sample_df = pd.DataFrame([], columns=columns)

    for label in labels:
        tmp = train_df[train_df['label'] == label]
        tmp_sampled = tmp.sample(frac=1).iloc[:count_min_label]
        train_sample_df = pd.concat([train_sample_df, tmp_sampled])

    train_sample_df = train_sample_df.sample(frac=1)

['seg', 'seg', 'snt']
10 4 10
10


# Model Preparation

In [3]:
def forward_func_ig(inputs):
    now = 0

    for embeddings, count in zip(inputs, sentence_counts):
        embeddings = embeddings.unsqueeze(0)
        embeddings = embeddings.swapaxes(0, 1)

        embeddings = model.pe(embeddings)

        src_mask, src_key_padding_mask = make_masks(model.max_len, count, device)

        encoder_output = model.encoder(embeddings, mask=src_mask, src_key_padding_mask=src_key_padding_mask)

        encoder_output = torch.mean(encoder_output[:count], dim=0)

        if now == 0:
            result_outputs = encoder_output
        else:
            result_outputs = torch.cat([result_outputs, encoder_output], dim=1)
        now += 1
    
    outputs = model.classifier1(result_outputs)
    logits = model.classifier2(outputs)
    return logits

In [4]:
def forward_func_ig2(inputs, sentence_counts):
    now = 0

    for encoder_output, count in zip(inputs, sentence_counts):
        encoder_output = encoder_output.swapaxes(0, 1)
        encoder_output = torch.mean(encoder_output[:count], dim=0)

        if now == 0:
            result_outputs = encoder_output
        else:
            result_outputs = torch.cat([result_outputs, encoder_output], dim=1)
        now += 1
        
    outputs = model.classifier1(result_outputs)
    logits = model.classifier2(outputs)
    return logits

ig = IntegratedGradients(forward_func_ig2)

In [5]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device('cpu')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

model_path = f'../predicting-satisfaction-using-graphs/splitbert/model/seg_seg_snt/epoch_4.model'
model = SplitBertConcatEncoderModel(num_labels=len(labels), embedding_size=384, max_len=max_count, device='cpu', pc_segmentation=False)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to('cpu')
model.eval()

for param in model.sbert.parameters():
    param.requires_grad = False
    
for param in model.bert.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
def construct_input_ref_pair(triplet):
    input_ids_list, ref_input_ids_list, attention_masks_list, sentence_count_list = [], [], [], []
    
    for contents in triplet:
        result = tokenizer(contents, pad_to_max_length=True, truncation=True, max_length=256, return_tensors='pt')
        
        input_ids = result['input_ids']
        sentence_count_list.append(torch.tensor(len(input_ids)).unsqueeze(0))
        attention_masks = result['attention_mask']
        
        pad = (0, 0, 0, max_count-len(input_ids))
        input_ids = nn.functional.pad(input_ids, pad, "constant", 0)
        attention_masks = nn.functional.pad(attention_masks, pad, "constant", 0)
        ref_input_ids = torch.zeros_like(input_ids)

        input_ids_list.append(input_ids.unsqueeze(0))
        ref_input_ids_list.append(ref_input_ids.unsqueeze(0))
        attention_masks_list.append(attention_masks.unsqueeze(0))
    
    return input_ids_list, ref_input_ids_list, attention_masks_list, sentence_count_list

In [7]:
def summarize_attributions(attribution):
    attributions = attribution.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [69]:
def splitbert_integrated_gradient(index, post, comment, reply, p_sentences, c_sentences, r_sentences, label, score, visualize=False):
    
    def post_or_comment_or_reply(index):
        for i, sentences in enumerate(all_sentences):
            if all_tokens[index] in sentences:
                if i == 0:
                    return 'post'
                elif i == 1:
                    return 'comment'
                else:
                    return 'reply'
    
    input_ids, ref_input_ids, attention_masks, sentence_counts = construct_input_ref_pair([post, comment, reply])
    
    
    one_hot_labels = torch.nn.functional.one_hot(torch.tensor(label), num_classes=len(labels))
    inputs = {'labels': one_hot_labels.type(torch.float).to(device),
          'input_ids1': input_ids[0].to(device),
          'input_ids2': input_ids[1].to(device),
          'input_ids3': input_ids[2].to(device),
          'attention_mask1': attention_masks[0].to(device),
          'attention_mask2': attention_masks[1].to(device),
          'attention_mask3': attention_masks[2].to(device),
          'sentence_count1': sentence_counts[0].to(device),
          'sentence_count2': sentence_counts[1].to(device),
          'sentence_count3': sentence_counts[2].to(device),
          'mode': 'triplet'
         }
    
    with torch.no_grad():
        embeddings = model(**inputs).hidden_states
        
    inputs = torch.stack(embeddings, dim=0)
    pred = forward_func_ig2(inputs, sentence_counts)
    # print(f'answer: {label}, predict: {torch.argmax(pred)}')
    
    result = []
    
    attribution, delta = ig.attribute(inputs=inputs, target=torch.argmax(pred), additional_forward_args=sentence_counts, return_convergence_delta=True)
    attributions = summarize_attributions(attribution)
    f_attributions = torch.flatten(attributions)
    f_attributions = f_attributions[f_attributions.nonzero()].squeeze(1)
    abs_attributions = list(map(abs, map(float, f_attributions)))
    idx_attributions = []
    for j in range(len(abs_attributions)):
        idx_attributions.append((j, abs_attributions[j], f_attributions[j].item()))
    idx_attributions.sort(key=lambda x:x[1], reverse=True)
        
    top3 = idx_attributions[:3]
    
    if visualize:
        all_sentences = [['[[post]]'], post, ['[[comment]]'], comment, ['[[reply]]'], reply]
        all_tokens = [item for all_sentences in all_sentences for item in all_sentences]
        
        vis_attributions = []
        
        j = 0
        for i in range(len(all_tokens)):
            if all_tokens[i] in ['[[post]]', '[[comment]]', '[[reply]]']:
                vis_attributions.append(0)
            else:
                vis_attributions.append(f_attributions[j].item())
                j += 1
                
        vis_attributions = torch.tensor(vis_attributions)
        
        score_vis = viz.VisualizationDataRecord(vis_attributions,
                                                torch.max(torch.softmax(pred, dim=0)),
                                                torch.argmax(pred),  # predicted label
                                                label,  # true label
                                                p_sentences + ' ' + c_sentences + ' ' + r_sentences,
                                                vis_attributions.sum(),
                                                all_tokens,
                                                delta)
        raw_text = ' '.join(post) + ' '.join(comment) + ' '.join(reply)
        
        print(vis_attributions)
        print('\033[1m', 'Visualization For Score', '\033[0m')
        viz.visualize_text([score_vis])
        
    else:
        where = []
        
        all_sentences = [post, comment, reply]
        all_tokens = [item for all_sentences in all_sentences for item in all_sentences]

        for j in range(len(top3)):
            where.append(post_or_comment_or_reply(top3[j][0]))

        result.append([index, post, comment, reply, score, all_tokens[top3[0][0]], top3[0][2], where[0]])
        result.append([index, post, comment, reply, score, all_tokens[top3[1][0]], top3[1][2], where[1]])
        result.append([index, post, comment, reply, score, all_tokens[top3[2][0]], top3[2][2], where[2]])

        return result, label, torch.argmax(pred).item()

In [9]:
train_index_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/train_index.csv', encoding='UTF-8')
val_index_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/val_index.csv', encoding='UTF-8')

train_index = sorted(list(train_index_df.idx.values))
val_index = sorted(list(val_index_df.idx.values))

In [10]:
print(len(train_index))
print(len(val_index))

546
100


In [56]:
def main(index_list):
    result_list = []
    label_pred_list = []
    
    for i in index_list:
        result, label, pred = splitbert_integrated_gradient(i, post_sequences[i], comment_sequences[i], reply_sequences[i], post_contents[i], comment_bodies[i], reply_contents[i], satisfactions[i], satisfactions_float[i])
        result_list.extend(result)
        
        for i in range(3):
            label_pred_list.append((label, pred))
        
    return result_list, label_pred_list   

In [75]:
if __name__ == "__main__":
    filtered = True
    
    result_list, label_pred_list = main(val_index)
    
    if filtered:
        filtered_list = []
        filtered_lp_list = []
        for result, lp in zip(result_list, label_pred_list):
            if abs(result[6]) >= 0.5:
                filtered_list.append(result)
                filtered_lp_list.append(lp)
    
    path = '../predicting-satisfaction-using-graphs/csv/integrated_gradient_result/'
    columns = ['idx', 'post_text', 'comment_text', 'reply_text', 'score', 'attr_sentence', 'attr_score', 'origin']
    
    def make_csv(target_list, label_pred_list):
        for i in range(3):
            for j in range(3):
                result = []
                for k in range(len(target_list)):
                    if label_pred_list[k] == (i, j):
                        result.append(target_list[k])
                result_df = pd.DataFrame(result, columns=columns)
                
                if filtered:
                    filename = f'label_{i}_pred_{j}_attribution_filtered.csv'
                else:
                    filename = f'label_{i}_pred_{j}_attribution.csv'
                
                result_df.to_csv(path + filename)
    
    
    if not filtered:
        make_csv(result_list, label_pred_list)
    else:
        make_csv(filtered_list, filtered_lp_list)

/home/mykim/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [216]:
for i in range(3):
    for j in range(3):
        print(f'label: {i}, pred: {j}')
        
        result_df = pd.read_csv(f'../predicting-satisfaction-using-graphs/csv/integrated_gradient_result/label_{i}_pred_{j}_attribution.csv', encoding='UTF-8')
        print(result_df.origin.value_counts())
        print()

label: 0, pred: 0
reply      24
comment    17
post       10
Name: origin, dtype: int64

label: 0, pred: 1
reply      5
comment    3
post       1
Name: origin, dtype: int64

label: 0, pred: 2
Series([], Name: origin, dtype: int64)

label: 1, pred: 0
reply      16
comment    13
post        7
Name: origin, dtype: int64

label: 1, pred: 1
reply      67
post       18
comment    17
Name: origin, dtype: int64

label: 1, pred: 2
reply      11
comment     6
post        1
Name: origin, dtype: int64

label: 2, pred: 0
reply      2
comment    1
Name: origin, dtype: int64

label: 2, pred: 1
reply      12
post        2
comment     1
Name: origin, dtype: int64

label: 2, pred: 2
reply      47
comment    12
post        7
Name: origin, dtype: int64



In [78]:
index_list = []

for i in range(3):
    for j in range(3):
        print(f'label: {i}, pred: {j}')
        
        result_df = pd.read_csv(f'../predicting-satisfaction-using-graphs/csv/integrated_gradient_result/label_{i}_pred_{j}_attribution_filtered.csv', encoding='UTF-8')
        print(result_df.origin.value_counts())
        
        if i == j:
            index_list.append(set(list(result_df.idx.values)))
        
        print()

label: 0, pred: 0
reply      16
comment     8
Name: origin, dtype: int64

label: 0, pred: 1
reply      3
comment    1
post       1
Name: origin, dtype: int64

label: 0, pred: 2
Series([], Name: origin, dtype: int64)

label: 1, pred: 0
reply      14
comment     2
Name: origin, dtype: int64

label: 1, pred: 1
reply      39
comment     5
Name: origin, dtype: int64

label: 1, pred: 2
reply    12
Name: origin, dtype: int64

label: 2, pred: 0
Series([], Name: origin, dtype: int64)

label: 2, pred: 1
reply    8
Name: origin, dtype: int64

label: 2, pred: 2
reply      37
comment     3
Name: origin, dtype: int64



In [82]:
for j in index_list[2]:
    splitbert_integrated_gradient(j, post_sequences[j], comment_sequences[j], reply_sequences[j], post_contents[N], comment_bodies[j], reply_contents[j], satisfactions[j], satisfactions_float[j], True)

tensor([ 0.0000,  0.1460,  0.0000, -0.0309,  0.0000,  0.8433,  0.5163])
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2 (1.00),"I'm wondering if anyone here has thought about suicide methods that come close to killing you but can't. I'm not ready to die but I want to hurt. I want people to see physically how much I'm hurting emotionally.I've thought about jumping in front of cars or falling from non-lethal heights. Those aren't the nicest methods but I'll go with them if I can't find a better one. Thank u too, for well,.... listening. Even tho its a small thing,for many of us ( specifically me) means a lot * Sincere pet on the shoulder* Ey, you're welcome. :)",1.47,"[[post]] I have been doing some experiments with being a bit extra open to my surroundings the last few years..But by getting to know new people lately I find myself misjudging my ""expertise"". [[comment]] Thank u too, for well,.... listening. Even tho its a small thing,for many of us ( specifically me) means a lot [[reply]] * Sincere pet on the shoulder* Ey, you're welcome. :)"


tensor([ 0.0000,  0.0348,  0.0421, -0.0080,  0.0095, -0.0431,  0.0000, -0.3536,
         0.0000,  0.4402,  0.2149,  0.4825,  0.6303])
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2 (1.00),"I'm wondering if anyone here has thought about suicide methods that come close to killing you but can't. I'm not ready to die but I want to hurt. I want people to see physically how much I'm hurting emotionally.I've thought about jumping in front of cars or falling from non-lethal heights. Those aren't the nicest methods but I'll go with them if I can't find a better one. I think you should look for the types of jobs that you think you can handle. I learned that I can't handle retail, and I probably can't handle restaurant work, so I'm looking for office jobs. Thank you for replying. I've often thought an office job would be a good fit for me too. I think I will try something of that nature. Thanks again.",1.45,"[[post]] I've been suffering with anxiety and depression for the past couple of years. Since April I have been dealing with bouts of derealization caused by anxiety. I quit my job in April and have yet to find a new job that I feel comfortable with. I went to therapy for a while and that helped a decent amount, offering me new insight on my problems. I worked on an assembly line two months ago for only two days before quitting. I am ashamed and feel like I am just being lazy, and letting my fear overcome my work ethic. I am wondering from all of your experiences, if you could offer advice or guidance towards a specific line of work that I should pursue to help me get back on my feet. I think restaurant work would destroy me. My old job was retail. I am 80% recovered in regards to the derealization. But I have a huge fear of it interrupting my life at unpredictable times. ANY advice would be much appreciated. Am I just a lazy asshole dreading work, or do you think this is related to the depression/anxiety? How do I push through it all and stay calm? [[comment]] I think you should look for the types of jobs that you think you can handle. I learned that I can't handle retail, and I probably can't handle restaurant work, so I'm looking for office jobs. [[reply]] Thank you for replying. I've often thought an office job would be a good fit for me too. I think I will try something of that nature. Thanks again."


tensor([ 0.0000,  0.1615,  0.0520,  0.1987,  0.0000,  0.0597, -0.0065,  0.1518,
         0.0000,  0.5438, -0.1622,  0.4657,  0.6051])
 Visualization For Score 


...

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2 (1.00),"I'm wondering if anyone here has thought about suicide methods that come close to killing you but can't. I'm not ready to die but I want to hurt. I want people to see physically how much I'm hurting emotionally.I've thought about jumping in front of cars or falling from non-lethal heights. Those aren't the nicest methods but I'll go with them if I can't find a better one. Iâm not trying to be rude or scare you at all. But no. It IS serious. Really, I wish and hope you will talk to your doctor about this immediately. Depression can Disassociate people. But it does not give them delusions.. Delusions derive from other physical or mental illness. And you need to address that so you canât treat it and stay healthy. Nah no worries. I was genuinley not aware it was this serious, thought it was just caused by the depression. I'll get the necessary help. Thank you.",1.39,"[[post]] Well yeah title says it all. The other night I thought there were eyes on every single surface in my room, but they'd close whenever I looked at them, and the eyelids were the actual surface, so they'd dissappear. Felt like my bed was gonna open up and swallow me whole. And yesterday and the day before I thought everyone in my life was some kind of actor, an alien, a robot, I'm not even sure. Like a Truman Show type thing. I thought I was the centre of some kind of experiment. Anyone else experience anything similar? [[comment]] Iâm not trying to be rude or scare you at all. But no. It IS serious. Really, I wish and hope you will talk to your doctor about this immediately. Depression can Disassociate people. But it does not give them delusions.. Delusions derive from other physical or mental illness. And you need to address that so you canât treat it and stay healthy. [[reply]] Nah no worries. I was genuinley not aware it was this serious, thought it was just caused by the depression. I'll get the necessary help. Thank you."


tensor([ 0.0000e+00, -2.5246e-02, -2.8836e-02, -1.7418e-04,  0.0000e+00,
        -1.5849e-01,  0.0000e+00,  7.6074e-01,  6.2824e-01])
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2 (1.00),"I'm wondering if anyone here has thought about suicide methods that come close to killing you but can't. I'm not ready to die but I want to hurt. I want people to see physically how much I'm hurting emotionally.I've thought about jumping in front of cars or falling from non-lethal heights. Those aren't the nicest methods but I'll go with them if I can't find a better one. If you could I would tell your manager how you've been feeling and if he should ever have trouble with the new hire, he consider you to come back. :) keep your chin up, new job or not. Thank you :)! I was thinking the same thing but, I wasn't sure if that would be weird to go back on my word.",1.18,"[[post]] I quit my job, after 4 months, while I was experiencing major anxiety and depression. I broke down (tears and all) to my boss and let him know how uncomfortable I've been working in his company. I'm a bookkeeper/admin at a design company and I'm totally isolated and work alone. Its made me feel worse and the quiet days dragged so much. Since ""I quit"" my managers have been really nice to me and showed more interest in me. Ive still stuck around because I felt bad about leaving empty staffed so I offered to train my replacement. My replacement is a total ditz and won't last in the fast pace environment we work in. I kind of want my job back because I didn't find another job yet and I'm starting to enjoy what I do. I'm afraid if my depression comes again I'll be at the same original state I started, feeling isolated and unmotivated to give work any effort. Anybody go through anything similar? [[comment]] If you could I would tell your manager how you've been feeling and if he should ever have trouble with the new hire, he consider you to come back. :) keep your chin up, new job or not. [[reply]] Thank you :)! I was thinking the same thing but, I wasn't sure if that would be weird to go back on my word."


tensor([ 0.0000, -0.0068,  0.0000, -0.0388,  0.0000,  0.7103,  0.7028])
 Visualization For Score 


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,2 (1.00),"I'm wondering if anyone here has thought about suicide methods that come close to killing you but can't. I'm not ready to die but I want to hurt. I want people to see physically how much I'm hurting emotionally.I've thought about jumping in front of cars or falling from non-lethal heights. Those aren't the nicest methods but I'll go with them if I can't find a better one. Im proud of you! For the first time in a couple days i managed to eat lunch, im hoping i can wake up early enough to eat breakfast soon. Good work though im proud and hope you keep up the good work! Thank you! I hope you will improve too!!!",1.37,"[[post]] I just managed to eat for over one week in a way that seems pretty healthy, and not like 100kcal a day max...I don't know how but I'm proud of it. Hope I won't relapse and gain some weight [[comment]] Im proud of you! For the first time in a couple days i managed to eat lunch, im hoping i can wake up early enough to eat breakfast soon. Good work though im proud and hope you keep up the good work! [[reply]] Thank you! I hope you will improve too!!!"
